In [2]:
import requests
from bs4 import BeautifulSoup

# get the links from the main page
url_root = "https://elections.public.lu"

r = requests.get(url_root + '/en')

In [3]:
# get the good parts with the name and url of each commune
soup = BeautifulSoup(r.text)
#print(soup.prettify())
coms = soup.find(id="communes").find_all("li", class_="town")

com_results = {}

for com in coms:
    #print(com.prettify())
    com_name = com.a.text
    com_url  = url_root + com.a['href']
    
    com_results[com_name] = {"url": com_url}

In [4]:
com_results

{'Beaufort': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/beaufort.html'},
 'Bech': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/bech.html'},
 'Beckerich': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/beckerich.html'},
 'Berdorf': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/berdorf.html'},
 'Bertrange': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/bertrange.html'},
 'Bettembourg': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/bettembourg.html'},
 'Bettendorf': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/bettendorf.html'},
 'Betzdorf': {'url': 'https://elections.public.lu/en/elections-legislatives/2023/resultats/communes/betzdorf.html'},
 'Bissen': {'url': 'https://elections.public.lu/en/elections

In [3]:
#cr = requests.get(com_results['Beaufort']['url'])

In [5]:
# Go through each commune
for com_name in com_results:
    print(com_name)
    
    # 'pass' by reference
    com = com_results[com_name]

    # get the page
    cr = requests.get(com['url'])
    csoup = BeautifulSoup(cr.text)

    # go through each section of the statistics
    for data_section in csoup.find_all("div", class_="lux-number"):
        heading = data_section.h3.text

        if heading == "Polling stations":
            stats = data_section.find_all("li")
            
            ps_count = int(stats[0].span.text.replace(' ', ''))
            ps_lv_only = int(stats[1].span.text.replace(' ', ''))
            ps_complete = int(stats[1].span.text.replace(' ', ''))

            com[heading] = {"ps_count": ps_count, "ps_lv_only": ps_lv_only, "ps_complete": ps_complete}

        if heading == "Candidates":
            stats = data_section.find_all("li")

            cand_total = int(stats[0].span.text.replace(' ', ''))
            cand_women = int(stats[1].span.text.replace(' ', ''))
            cand_men = int(stats[2].span.text.replace(' ', ''))

            com[heading] = {"cand_women": cand_women, "cand_men": cand_men, "cand_total": cand_total}

        if heading == "Electorate":
            stats = data_section.find_all("li")

            el_registered = int(stats[0].span.text.replace(' ', ''))
            el_postal_ballots = int(stats[1].span.text.replace(' ', ''))
            
            com[heading] = {"el_registered": el_registered, "el_postal_ballots": el_postal_ballots}

        if heading == "Ballots":
            stats = data_section.find_all("li")

            bl_in_box = int(stats[0].span.text.replace(' ', ''))
            bl_valid = int(stats[1].span.text.replace(' ', ''))
            bl_blank = int(stats[2].span.text.replace(' ', ''))
            bl_invalid = int(stats[3].span.text.replace(' ', ''))
            bl_postal = int(stats[4].span.text.replace(' ', ''))

            com[heading] = {"bl_in_box": bl_in_box, "bl_valid": bl_valid, "bl_blank": bl_blank,
                            "bl_invalid": bl_invalid, "bl_postal": bl_postal}

        if heading == "Votes":
            stats = data_section.find_all("li")

            vot_count = int(stats[0].span.text.replace(' ', ''))
            vot_total_cast = int(stats[1].span.text.replace(' ', ''))

            com[heading] = {"vot_count": vot_count, "vot_total_cast": vot_total_cast}
            
        # end of sections
        
    # end of commune
    

Beaufort
Bech
Beckerich
Berdorf
Bertrange
Bettembourg
Bettendorf
Betzdorf
Bissen
Biwer
Boulaide
Bourscheid
Bous-Waldbredimus
Clervaux
Colmar-Berg
Consdorf
Contern
Dalheim
Diekirch
Differdange
Dippach
Dudelange
Echternach
Ell
Erpeldange-sur-Sûre
Esch-sur-Alzette
Esch-sur-Sûre
Ettelbruck
Feulen
Fischbach
Flaxweiler
Frisange
Garnich
Goesdorf
Grevenmacher
Grosbous-Wahl
Heffingen
Helperknapp
Hesperange
Hobscheid
Junglinster
Käerjeng
Kayl
Kehlen
Kiischpelt
Koerich
Kopstal
Lac de la Haute-Sûre
Larochette
Lenningen
Leudelange
Lintgen
Lorentzweiler
Luxembourg
Mamer
Manternach
Mersch
Mertert
Mertzig
Mondercange
Mondorf-les-Bains
Niederanven
Nommern
Parc Hosingen
Pétange
Preizerdaul
Putscheid
Rambrouch
Reckange-sur-Mess
Redange/Attert
Reisdorf
Remich
Roeser
Rosport-Mompach
Rumelange
Saeul
Sandweiler
Sanem
Schengen
Schieren
Schifflange
Schuttrange
Stadtbredimus
Steinfort
Steinsel
Strassen
Tandel
Troisvierges
Useldange
Vallée de l'Ernz
Vianden
Vichten
Waldbillig
Walferdange
Weiler-la-Tour
Weiswampa

In [6]:
import json

json_object = json.dumps(com_results, indent=4)
with open("2023_legislative_election_results.json", "w") as fh:
    fh.write(json_object)

In [7]:
with open("2023_legislative_election_results.csv", "w") as fh:
    #print(com_results)
    header = True
    headerline = ["com_name"]
    
    for com_name in com_results:
        # KIS
        if 'url' in com_results[com_name]:
            del com_results[com_name]['url']

        combined = [com_name]
        for section in com_results[com_name]:
            if header:
                headerline += com_results[com_name][section]
                
            combined += com_results[com_name][section].values()
            
        if header:
            header = False
            fh.write(','.join(headerline) + '\n')
        fh.write(','.join(str(x) for x in combined) + '\n')
